#### Postgres Connection

In [76]:
# !pip install SQLAlchemy
# # !pip install tabulate

import psycopg2
from openai import OpenAI
from dotenv import load_dotenv
import os
import pandas as pd
from util.const import *

load_dotenv()

HOST = os.getenv('DB_HOST')
PORT = os.getenv('DB_PORT')
USER = os.getenv('DB_USER')
PASSWORD = os.getenv('DB_PASSWORD')
DATABASE = os.getenv('DB_NAME')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [77]:
try:
    conn = psycopg2.connect(
        host=HOST,
        port=PORT,
        user=USER,
        password=PASSWORD,
        database=DATABASE
    )
    print("Database connection successful")
    
    conn.close()

except psycopg2.Error as e:
    print(f"Error connecting to PostgreSQL: {e}")

Database connection successful


##### pg_vector, PostGIS extenstion

In [4]:
try:
    conn = psycopg2.connect(
        host=HOST,
        port=PORT,
        user=USER,
        password=PASSWORD,
        database=DATABASE
    )
    cur = conn.cursor()

    # PostGIS와 pgvector 확장 설치
    cur.execute("CREATE EXTENSION IF NOT EXISTS postgis;")
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")

    conn.commit()
    print("PostGIS 및 pgvector 확장이 성공적으로 설치되었습니다.")

except psycopg2.Error as e:
    print(f"작업 중 오류가 발생했습니다: {e}")
    
finally:
    if conn:
        cur.close()
        conn.close()

PostGIS 및 pgvector 확장이 성공적으로 설치되었습니다.


In [78]:
conn = psycopg2.connect(
    host=HOST,
    port=PORT,
    user=USER,
    password=PASSWORD,
    database=DATABASE
)
cur = conn.cursor()

cur.execute("SELECT * FROM pg_extension;")
res = cur.fetchall()

for data in res:
    print(data[1])
        
conn.commit()

cur.close()
conn.close()

plpgsql
vector
postgis


##### Geospatial Data Insertion (Newyork)

사실 실제 프로젝트에서는 이미 데이터베이스 데이터가 저장되어있기 때문에 insertion 단계가 필요없음.

In [ ]:
import json

json_file_path = './data/citibike_stations_sample.json'
with open(json_file_path, 'r') as file:
    data = json.load(file)
    
print(data)

In [109]:
from sqlalchemy import create_engine, text
import json
from tabulate import tabulate

json_file_path = './data/citibike_stations_sample.json'
with open(json_file_path, 'r') as file:
    data = json.load(file)

# SQLAlchemy 엔진 생성
DATABASE_URI =f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}'
engine = create_engine(DATABASE_URI)

# query = "SELECT * FROM citibike_stations LIMIT 5;"
# df = pd.read_sql(query, engine)

with engine.connect() as conn:
    # conn.execute(text(create_table_query))
    # conn.execute(text(insert_query), data)
    select_query = '''
SELECT 
    ST_AsText(ST_MakePoint(longitude, latitude)) AS WKT
FROM 
    citibike_stations
WHERE 
    name = 'Leonard Gordon Park'
'''
    result = conn.execute(text(select_query))
    rows = result.fetchall()
    headers = result.keys()

wkt_point = rows[0][0]
print(wkt_point)
# tabulate로 결과 출력
table_string = tabulate(rows, headers, tablefmt="psql")

POINT(-74.0572714805603 40.74590996631558)


In [104]:
from sqlalchemy import create_engine, text
import pandas as pd
import json

# SQLAlchemy 엔진 생성
DATABASE_URI = f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}'
engine = create_engine(DATABASE_URI)

# SELECT 쿼리 (citibike_stations 테이블의 스키마 정보 가져오기)
query = '''
SELECT
    column_name,
    data_type,
    is_nullable
FROM
    INFORMATION_SCHEMA.COLUMNS
WHERE
    table_name = 'citibike_stations'
'''

# pandas를 사용하여 SQL 실행 후 DataFrame으로 변환
df = pd.read_sql(query, engine)

# DataFrame을 JSON으로 변환
json_output = df.to_json(orient='records', indent=2)

# JSON 결과 출력
print(json_output)

[
  {
    "column_name":"last_reported",
    "data_type":"timestamp without time zone",
    "is_nullable":"YES"
  },
  {
    "column_name":"is_installed",
    "data_type":"boolean",
    "is_nullable":"YES"
  },
  {
    "column_name":"is_renting",
    "data_type":"boolean",
    "is_nullable":"YES"
  },
  {
    "column_name":"is_returning",
    "data_type":"boolean",
    "is_nullable":"YES"
  },
  {
    "column_name":"eightd_has_available_keys",
    "data_type":"boolean",
    "is_nullable":"YES"
  },
  {
    "column_name":"latitude",
    "data_type":"double precision",
    "is_nullable":"YES"
  },
  {
    "column_name":"longitude",
    "data_type":"double precision",
    "is_nullable":"YES"
  },
  {
    "column_name":"region_id",
    "data_type":"bigint",
    "is_nullable":"YES"
  },
  {
    "column_name":"capacity",
    "data_type":"bigint",
    "is_nullable":"YES"
  },
  {
    "column_name":"eightd_has_key_dispenser",
    "data_type":"boolean",
    "is_nullable":"YES"
  },
  {
    "col

In [105]:
wkt_point

'POINT(-74.0572714805603 40.74590996631558)'

In [82]:
print(table_string)

+--------------------------------------------+
| wkt_location                               |
|--------------------------------------------|
| POINT(-74.0572714805603 40.74590996631558) |
+--------------------------------------------+


In [55]:
########## bigquery와 직접 연동하는 방법 (지금은 .csv 파일로 데이터 저장해둬서 불필요) ########## 
# !pip install google-cloud-bigquery
# !pip install db-dtypes

import glob
from google.cloud import bigquery
from google.oauth2 import service_account

# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("./config/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# BigQuery 클라이언트 설정
client = bigquery.Client(credentials = credentials, project = credentials.project_id)

#### OpenAI Connection

In [41]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = OpenAI(api_key=OPENAI_API_KEY)
prompt = f'hi'

response = llm.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=0
    )

print('Answer:', response.choices[0].message.content)

Answer: Hello! How can I assist you today?


In [90]:
pip install shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


#### Folium Tutorial

In [99]:
type(wkt_point)

str

In [103]:
from shapely import wkt
import folium

point = wkt.loads(wkt_point)

coordinates = [point.y, point.x]
# print(coordinates)

mymap = folium.Map(location=coordinates, zoom_start=15)
folium.Marker(location=coordinates, popup=f"WKT Point").add_to(mymap)


mymap

[40.74590996631558, -74.0572714805603]


In [2]:
import json

with open('/Users/kunwooshin/Desktop/askgeo/data/metadata/spatialdb_desc.json', 'r') as file:
    data = json.load(file)

In [5]:
data

[{'table_name': 'citibike_stations',
  'table_description': 'The New York City Citi Bike Stations dataset provides detailed information about Citi Bike stations across the city, including their locations, statuses, and availability.'}]